In [ ]:
import pandas as pd

df = pd.read_json("../data/dev/dev.json")

df.head()

,question_id,db_id,question,evidence,SQL,difficulty
0,0,california_schools,What is the highest eligible free rate for K-1...,Eligible free rate for K-12 = `Free Meal Count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,simple
1,1,california_schools,Please list the lowest three eligible free rat...,Eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,moderate
2,2,california_schools,Please list the zip code of all the charter sc...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,simple
3,3,california_schools,What is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,simple
4,4,california_schools,Please list the phone numbers of the direct ch...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,moderate


In [8]:
percentages = df["difficulty"].value_counts(normalize=True) * 100

print(percentages)

difficulty
simple         60.299870
moderate       30.247718
challenging     9.452412
Name: proportion, dtype: float64


In [36]:
sample = df.sample(100 , random_state=0)

sample_distribution = sample["difficulty"].value_counts(normalize=True) * 100

sample_distribution

difficulty
simple         63.0
moderate       30.0
challenging     7.0
Name: proportion, dtype: float64

In [ ]:
os.makedirs("../sample", exist_ok=True)

sample.to_csv("../sample/dev.csv", index=False)